In [1]:
# For enabling running in google colab

# !pip3 install pyDOE
# !pip3 install complexPyTorch
# !pip3 install cplxmodule

# import sys; sys.path.insert(1, '/content/drive/MyDrive/Colab Notebooks/')
# import os
# from glob import glob
# from google.colab import drive
# drive.mount('/content/drive')
# PATH = '/content/drive/MyDrive/Colab Notebooks/'
# DATA_PATH = PATH + 'data/NLS.mat'
# from helper.utils import *
# from helper.models import *

In [2]:
import numpy as np
import scipy.io as io
from pyDOE import lhs
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from complexPyTorch.complexLayers import ComplexLinear

import cplxmodule
# complex valued tensor class
from cplxmodule import cplx
from cplxmodule.nn import RealToCplx, CplxToReal, CplxSequential, CplxToCplx
from cplxmodule.nn import CplxLinear, CplxModReLU, CplxAdaptiveModReLU

from utils import *
from models import *

In [3]:
# torch device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You're running on", device)

# Doman bounds
lb = np.array([-5.0, 0.0])
ub = np.array([5.0, np.pi/2])

N0 = 50
N_b = 50
N_f = 20000

DATA_PATH = './experimental_data/NLS.mat'
data = io.loadmat(DATA_PATH)

t = data['tt'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = data['uu']
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
Exact_h = np.sqrt(Exact_u**2 + Exact_v**2)
h_star = Exact_h.T.flatten()[:,None]

idx_x = np.random.choice(x.shape[0], N0, replace=False)
x0 = x[idx_x, :]
Exact_u0 = np.real(Exact)[idx_x, 0:1]
Exact_v0 = np.imag(Exact)[idx_x, 0:1]
h0 = np.hstack((Exact_u0, Exact_v0))

idx_t = np.random.choice(t.shape[0], N_b, replace=False)
tb = t[idx_t,:]

X_f = lb + (ub-lb)*lhs(2, N_f) # -> PDE Loss
X0 = np.concatenate((x0, 0*x0), 1) # (x0, 0)
X_lb = np.concatenate((0*tb + lb[0], tb), 1) # (lb[0], tb)
X_ub = np.concatenate((0*tb + ub[0], tb), 1) # (ub[0], tb)

# PDE Loss
X_f = to_tensor(X_f, True).to(device)

# NN(X0) approx. h0
X0 = to_tensor(X0, False).to(device)
h0 = to_tensor(h0, False).to(device)

# NN(X_lb) approx NN(X_ub)
# NN(X_lb)_x approx NN(X_ub)_x, _x = diff wrt the first dimension
X_lb = to_tensor(X_lb, True).to(device)
X_ub = to_tensor(X_ub, True).to(device)

lb = to_tensor(lb, False).to(device)
ub = to_tensor(ub, False).to(device)

X_star = to_tensor(X_star, False).to(device)
# h_star = to_complex_tensor(h_star, False).to(device)

In [4]:
class Act(cnn.CplxToCplx):
    def __init__(self,):
        super(Act, self).__init__()
        pass
    def forward(self, inp):
        return torch.tanh(cplx2tensor(inp))
    
class ImaginaryDimensionAdder(nn.Module):
    def __init__(self,):
        super(ImaginaryDimensionAdder, self).__init__(); pass
    def forward(self, real_tensor):
        added = cat(real_tensor[:, 0:1], torch.zeros(real_tensor.shape[0], 1))
        for i in range(1, real_tensor.shape[1]):
            added = cat(added, real_tensor[:, i:i+1], torch.zeros(real_tensor.shape[0], 1))
        return added
        
# act = Act
# act = CplxAdaptiveModReLU
inp_dimension = 2
act = CplxToCplx[torch.tanh]
complex_model = CplxSequential(
    CplxLinear(inp_dimension, 100, bias=True),
    act(),
    CplxLinear(100, 100, bias=True),
    act(),
    CplxLinear(100, 100, bias=True),
    act(),
    CplxLinear(100, 100, bias=True),
    act(),
    CplxLinear(100, 1, bias=True),
)

/usr/local/lib/python3.9/site-packages/torch/nn/modules/container.py:587: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")


In [5]:
m = nn.Sequential(
        ImaginaryDimensionAdder(),
        RealToCplx(),
        complex_model,
#         CplxToReal(),
    )

In [6]:
class ComplexPhysicsInformedNN(nn.Module):
    def __init__(self, model, lb, ub, scale=False):
        super(ComplexPhysicsInformedNN, self).__init__()
        self.model = model
        self.lb = lb
        self.ub = ub
        self.scale = scale
    
    def forward(self, X):
        if self.scale: 
            return self.model(self.neural_net_scale(X))
        return self.model(X)

    def predict(self, X_test):
        return CplxToReal()(self.forward(self.preprocess(*dimension_slicing(X_test))))
    
    def neural_net_scale(self, inp):
        return (2.0*(inp-self.lb)/(self.ub-self.lb))-1.0

    def preprocess(self, spatial, time):
        return cat(spatial, time)
    
    def loss(self, X_f, X0, h0, X_lb, X_ub):
        loss = self.net_f(*dimension_slicing(X_f))
        h0_pred = self.predict(X0); u0 = h0_pred[:, 0:1]; v0 = h0_pred[:, 1:2]
        loss += F.mse_loss(u0, h0[:, 0:1])+F.mse_loss(v0, h0[:, 1:2])
        u_lb, v_lb, u_lb_x, v_lb_x = self.net_h(*dimension_slicing(X_lb))
        u_ub, v_ub, u_ub_x, v_ub_x = self.net_h(*dimension_slicing(X_ub))
        loss += F.mse_loss(u_lb, u_ub)
        loss += F.mse_loss(v_lb, v_ub)
        loss += F.mse_loss(u_lb_x, u_ub_x)
        loss += F.mse_loss(v_lb_x, v_ub_x)
        return loss
    
    def net_h(self, x, t):
        X = cat(x, t)
        h = self.forward(X)
        u = h.real
        v = h.imag
        return u, v, self.diff(u, x), self.diff(v, x)
    
    def net_f(self, x, t):
        u, v, u_x, v_x = self.net_h(x, t)
        u_t, v_t = self.diff(u, t), self.diff(v, t)
        u_xx, v_xx = self.diff(u_x, x), self.diff(v_x, x)
        f_u = u_t + 0.5*v_xx + (u**2 + v**2)*v
        f_v = v_t - 0.5*u_xx - (u**2 + v**2)*u
        return (f_u**2).mean()+(f_v**2).mean()

    def diff(self, func, inp):
        return grad(func, inp, create_graph=True, retain_graph=True, grad_outputs=torch.ones(func.shape, dtype=func.dtype).to(device))[0]

    def complex_mse(self, v1, v2):
        assert v1.shape == v2.shape
        assert v1.shape[1] == 1
        return F.mse_loss(v1.real, v2.real)+F.mse_loss(v2.imag, v2.imag)

    def add_imag_dim(self, v1):
        z = torch.zeros(v1.shape).requires_grad_(False).to(device)
        return torch.complex(v1, z)

In [7]:
layers = [2, 100, 100, 100, 100, 1]
model = TorchComplexMLP(layers)
cpinn = ComplexPhysicsInformedNN(model=m, lb=lb, ub=ub, scale=False).to(device)

In [8]:
# def add_imag_dim(v1):
#   z = torch.zeros(v1.shape).requires_grad_(False).to(device)
#   return torch.complex(v1, z)

# def tdiff(func, inp):
#   return grad(func, inp, create_graph=True, retain_graph=True, grad_outputs=torch.ones(func.shape, dtype=func.dtype).to(device))[0]

# x, t = dimension_slicing(X_f)
# a, b = add_imag_dim(x), add_imag_dim(t)
# func = model(cat(a, b))
# tdiff(func.real, t)

In [9]:
best_state_dict = None; best_loss = 1000.0
learning_rate1, learning_rate2 = 1e-3, 1e-1
optimizer1 = torch.optim.Adam(cpinn.parameters(), lr=learning_rate1)
epochs1 = 1000
print("1st Stage optimization")
for i in range(epochs1):
    cpinn.train()
    optimizer1.zero_grad()
    loss = cpinn.loss(X_f, X0, h0, X_lb, X_ub)
    loss.backward(retain_graph=False)
    optimizer1.step()

    track = loss.item()
    if track < best_loss:
      best_loss = track
      best_state_dict = cpinn.state_dict()

    if i%10==0: 
      cpinn.eval()
      print(track)
      h_star_pred = cpinn.predict(X_star)
      h_star_pred = torch.sqrt(h_star_pred[:, 0:1]**2 + h_star_pred[:, 1:2]**2)
      h_star_pred = (h_star_pred.detach().cpu().numpy())
      print('Test score', mean_squared_error(h_star_pred, h_star))

1st Stage optimization
4.6230244636535645
Test score 0.650792658358732
1.516767144203186
Test score 0.637255628148042
0.4470001757144928
Test score 0.38790888955214015
0.4434736967086792
Test score 0.3515484679506801
0.3543202877044678
Test score 0.3445935947824777
0.30723142623901367
Test score 0.32058153904209896
0.2450341433286667
Test score 0.2691525724699435
0.19877596199512482
Test score 0.23350022649095842
0.18728980422019958
Test score 0.2102306620028725
0.14753946661949158
Test score 0.1845154373208915
0.1359558403491974
Test score 0.17411084423822376
0.12862499058246613
Test score 0.17026527523281149
0.12386464327573776
Test score 0.16799392200253402
0.13405485451221466
Test score 0.16969658058343734
0.1481952965259552
Test score 0.16786675706378817
0.11653044819831848
Test score 0.16724235252911387
0.11698071658611298
Test score 0.16533738237580153
0.11031896620988846
Test score 0.16287644742867283
0.10831142216920853
Test score 0.1622804010094372
0.10557455569505692
Test sc

KeyboardInterrupt: 

In [27]:
# if best_state_dict is not None: cpinn.load_state_dict(best_state_dict)

optimizer2 = torch.optim.LBFGS(cpinn.parameters(), lr=learning_rate2, 
                               max_iter=200, max_eval=200, 
                               history_size=120, line_search_fn='strong_wolfe')

epochs2 = 50000
cpinn.train()
print("2st Stage optimization")

for i in range(epochs1):
    def closure():
      if torch.is_grad_enabled(): optimizer2.zero_grad()
      loss = cpinn.loss(X_f, X0, h0, X_lb, X_ub)
      if loss.requires_grad: loss.backward(retain_graph=False)
      return loss
    optimizer2.step(closure)

    track = closure().item()
    if track < best_loss:
      best_loss = track
      best_state_dict = cpinn.state_dict()

    if i%10==0: 
      cpinn.eval()
      print(track)
      h_star_pred = cpinn.predict(X_star)
      h_star_pred = torch.sqrt(h_star_pred[:, 0:1]**2 + h_star_pred[:, 1:2]**2)
      h_star_pred = (h_star_pred.detach().cpu().numpy())
      print('Test score', mean_squared_error(h_star_pred, h_star))

2st Stage optimization
0.0005419803783297539
Test score 0.0018069956771203238
0.00015163954230956733
Test score 0.00030664691319947805
7.657348032807931e-05
Test score 0.00011338522830886114
4.5973127271281555e-05
Test score 5.549353836509127e-05
3.10495997837279e-05
Test score 1.8905030561149737e-05
3.10495997837279e-05
Test score 1.8905030561149737e-05
3.10495997837279e-05
Test score 1.8905030561149737e-05
3.10495997837279e-05
Test score 1.8905030561149737e-05
3.10495997837279e-05
Test score 1.8905030561149737e-05
3.10495997837279e-05
Test score 1.8905030561149737e-05
3.10495997837279e-05
Test score 1.8905030561149737e-05
3.10495997837279e-05
Test score 1.8905030561149737e-05
3.10495997837279e-05
Test score 1.8905030561149737e-05
3.10495997837279e-05
Test score 1.8905030561149737e-05
3.10495997837279e-05
Test score 1.8905030561149737e-05
3.10495997837279e-05
Test score 1.8905030561149737e-05
3.10495997837279e-05
Test score 1.8905030561149737e-05
3.10495997837279e-05
Test score 1.8905

KeyboardInterrupt: ignored

In [28]:
h_star_pred = cpinn.predict(X_star)
h_star_pred = torch.sqrt(h_star_pred[:, 0:1]**2 + h_star_pred[:, 1:2]**2)
h_star_pred = (h_star_pred.detach().cpu().numpy())

print('Test score', mean_squared_error(h_star_pred, h_star))

Test score 1.8905041546390686e-05


In [30]:
np.linalg.norm(h_star-h_star_pred,2)/np.linalg.norm(h_star,2)

0.0048614245690865995